In [103]:
# Dependencies
import pandas as pd

In [104]:
# path file
filepath = "Resources/purchase_data.csv"

In [105]:
# Create dataframe
rawdata_df = pd.read_csv(filepath, encoding="ISO-8859-1")
copydata = rawdata_df.copy()
copydata.head()

,Purchase ID,SN,Age,Gender,Item ID,Item Name,Price
0,0,Lisim78,20,Male,108,"Extraction, Quickblade Of Trembling Hands",3.53
1,1,Lisovynya38,40,Male,143,Frenzied Scimitar,1.56
2,2,Ithergue48,24,Male,92,Final Critic,4.88
3,3,Chamassasya86,24,Male,100,Blindscythe,3.27
4,4,Iskosia90,23,Male,131,Fury,1.44


In [106]:
### Player Count
total = len(copydata["SN"].value_counts())
totalplayers = pd.DataFrame({"Total Players": [total]})
totalplayers

,Total Players
0,576


In [107]:
### Purchasing Analysis (Total)

unique_item = copydata["Item ID"].drop_duplicates().count()
avg_price = copydata["Price"].mean()
totalpurchases = copydata["Purchase ID"].count() 
revenue = copydata["Price"].sum()# 

pa_df = pd.DataFrame({"Unique Items": [unique_item],
                                      "Average Price" : [avg_price],
                                      "Total Purchases" : [totalpurchases],
                                      "Revenue":[revenue]})
pa_df["Average Price"] = pa_df["Average Price"].map("${:.2f}".format)
pa_df["Revenue"] = pa_df["Revenue"].map("${:,.2f}".format)
pa_df

,Unique Items,Average Price,Total Purchases,Revenue
0,179,$3.05,780,"$2,379.77"


In [108]:
# ### Gender Demographics
gd = copydata.groupby("Gender")
gdcount = gd.nunique()["SN"]
gcp = (gdcount / total) * 100
gc_df = pd.DataFrame({"Total Count":gdcount,"Percentage":gcp})
gc_df["Percentage"] = gc_df["Percentage"].map("{:.2f}%".format)
gc_df

,Total Count,Percentage
Gender,,
Female,81,14.06%
Male,484,84.03%
Other / Non-Disclosed,11,1.91%


In [109]:
# ### Purchasing Analysis (Gender)
gendergrouped = copydata.groupby(["Gender"])
gendercount = gendergrouped["Purchase ID"].count()
genderavg = gendergrouped["Price"].mean()
gendertotal = gendergrouped["Price"].sum()
avgpp = gendertotal/gendergrouped.nunique()["SN"]

pag_df = pd.DataFrame({"Purchase Count": gendercount,
                      "Average Purchase Price": genderavg,
                      "Total Purchase Value": gendertotal,
                      "Average Purchase Total per Person": avgpp})

pag_df["Average Purchase Price"] = pag_df["Average Purchase Price"].map("${:.2f}".format)
pag_df["Total Purchase Value"] = pag_df["Total Purchase Value"].map("${:,.2f}".format)
pag_df["Average Purchase Total per Person"] = pag_df["Average Purchase Total per Person"].map("${:,.2f}".format)

pag_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Gender,,,,
Female,113,$3.20,$361.94,$4.47
Male,652,$3.02,"$1,967.64",$4.07
Other / Non-Disclosed,15,$3.35,$50.19,$4.56


In [110]:
agebins = [0, 9, 14, 19, 24, 29, 34, 39, 200]
agelabels = ["Under 10", "10-14", "15-19", "20-24", "25-29", "30-34", "35-40", "Above 40"]
copydata["Age Group"] = pd.cut(copydata["Age"],agebins,labels=agelabels, include_lowest=True)
agegrouped = copydata.groupby("Age Group")
agecount = agegrouped["SN"].nunique()
ageperc = (agecount/total)
age_demo = pd.DataFrame({"Total Count":agecount,
            "Percentage of Players":ageperc})
age_demo["Percentage of Players"]=age_demo["Percentage of Players"].map("{:,.2%}".format)
age_demo

,Total Count,Percentage of Players
Age Group,,
Under 10,17,2.95%
10-14,22,3.82%
15-19,107,18.58%
20-24,258,44.79%
25-29,77,13.37%
30-34,52,9.03%
35-40,31,5.38%
Above 40,12,2.08%


In [111]:
###### ### Age Demographics

# * The below each broken into bins of 4 years (i.e. &lt;10, 10-14, 15-19, etc.)
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
#   * Average Purchase Total per Person by Age Group
agepurchcount = agegrouped["Purchase ID"].count()
ageavgprice = agegrouped["Price"].mean()
agetotalval = agegrouped["Price"].sum()
avgagetpp = agetotalval/agecount

pd_df = pd.DataFrame({"Purchase Count": agepurchcount,
                               "Average Purchase Price": ageavgprice,
                               "Total Purchase Value": agetotalval,
                               "Average Purchase Total per Person": avgagetpp})

pd_df["Average Purchase Price"] = pd_df["Average Purchase Price"].map("${:.2f}".format)
pd_df["Total Purchase Value"] = pd_df["Total Purchase Value"].map("${:,.2f}".format)
pd_df["Average Purchase Total per Person"] = pd_df["Average Purchase Total per Person"].map("${:,.2f}".format)
pd_df

,Purchase Count,Average Purchase Price,Total Purchase Value,Average Purchase Total per Person
Age Group,,,,
Under 10,23,$3.35,$77.13,$4.54
10-14,28,$2.96,$82.78,$3.76
15-19,136,$3.04,$412.89,$3.86
20-24,365,$3.05,"$1,114.06",$4.32
25-29,101,$2.90,$293.00,$3.81
30-34,73,$2.93,$214.00,$4.12
35-40,41,$3.60,$147.67,$4.76
Above 40,13,$2.94,$38.24,$3.19


In [112]:
# ### Top Spenders
# * Identify the the top 5 spenders in the game by total purchase value, then list (in a table):
#   * SN
#   * Purchase Count
#   * Average Purchase Price
#   * Total Purchase Value
spendinfo = copydata.groupby("SN")
purchasecountspender = spendinfo["Purchase ID"].count()
avgppspender = spendinfo["Price"].mean()
purchasetotalspender = spendinfo["Price"].sum()
topspenders_df = pd.DataFrame({"Purchase Count":purchasecountspender,
                            "Average Purchase Price":avgppspender,
                            "Total Purchase Value": purchasetotalspender})
topspenders_df.head()

sortedspenders_df = topspenders_df.sort_values(["Total Purchase Value"], ascending=False).head()
sortedspenders_df["Average Purchase Price"] = sortedspenders_df["Average Purchase Price"].map("${:.2f}".format)
sortedspenders_df["Total Purchase Value"] = sortedspenders_df["Total Purchase Value"].map("${:,.2f}".format)
sortedspenders_df

,Purchase Count,Average Purchase Price,Total Purchase Value
SN,,,
Lisosia93,5,$3.79,$18.96
Idastidru52,4,$3.86,$15.45
Chamjask73,3,$4.61,$13.83
Iral74,4,$3.40,$13.62
Iskadarya95,3,$4.37,$13.10


In [137]:
# ### Most Popular Items
                           
# * Identify the 5 most popular items by purchase count, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value

iteminfo= rawdata_df[["Item ID","Item Name","Price"]]
mpi_df = iteminfo.groupby(["Item ID", "Item Name"])
itempurchasecount = mpi_df["Price"].count()
popitemsum = mpi_df["Price"].sum()
popitemprice = popitemsum/itempurchasecount
cleanmpi_df = pd.DataFrame({"Purchase Count": itempurchasecount,
                           "Item Price": popitemprice,
                            "Total Purchase Value":popitemsum})

cleanmpi_df["Item Price"] = cleanmpi_df["Item Price"].map("${:.2f}".format)
cleanmpi_df["Total Purchase Value"] = cleanmpi_df["Total Purchase Value"].map("${:,.2f}".format)
sortedcleanmpi_df= cleanmpi_df.sort_values("Purchase Count", ascending = False)
sortedcleanmpi_df.head()

,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
92,Final Critic,13,$4.61,$59.99
178,"Oathbreaker, Last Hope of the Breaking Storm",12,$4.23,$50.76
145,Fiery Glass Crusader,9,$4.58,$41.22
132,Persuasion,9,$3.22,$28.99
108,"Extraction, Quickblade Of Trembling Hands",9,$3.53,$31.77


In [135]:
# ### Most Profitable Items

# * Identify the 5 most profitable items by total purchase value, then list (in a table):
#   * Item ID
#   * Item Name
#   * Purchase Count
#   * Item Price
#   * Total Purchase Value
most_profit = cleanmpi_df.sort_values("Total Purchase Value", ascending=False)
most_profit.head()


,,Purchase Count,Item Price,Total Purchase Value
Item ID,Item Name,,,
63,Stormfury Mace,2,$4.99,$9.98
29,"Chaos, Ender of the End",5,$1.98,$9.90
173,Stormfury Longsword,2,$4.93,$9.86
38,"The Void, Vengeance of Dark Magic",4,$2.37,$9.48
143,Frenzied Scimitar,6,$1.56,$9.36


In [ ]:
# As final considerations:

# * You must use the Pandas Library and the Jupyter Notebook.
# * You must submit a link to your Jupyter Notebook with the viewable Data Frames.
# * You must include a written description of three observable trends based on the data.
#  * See [Example Solution](HeroesOfPymoli/HeroesOfPymoli_starter.ipynb) for a reference on expected format.

In [143]:
analysis = ("Pymoli Analysis: Our users are predominantly male (84%) and most are within the 20-24 year old age range.Our most popula ritems are Final Critic and Oathbreaker.")

print(analysis)

Pymoli Analysis: Our users are predominantly male (84%) and most are within the 20-24 year old age range.Our most popula ritems are Final Critic and Oathbreaker.
